In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import random
from collections import OrderedDict
import math

In [3]:
#Recursive function to generate Hadamard matrices of order equal to powers of two

H = {}
H[1] = torch.tensor([[1.0]])
def Had(n):
    if(n not in H):
        Hnby2 = Had(n//2)
        Hn = torch.ones(n, n)
        for i in range(n//2):
            for j in range(n//2):
                Hn[i][j] = Hnby2[i][j]
                Hn[i+n//2][j] = Hnby2[i][j]
                Hn[i][j+n//2] = Hnby2[i][j]
                Hn[i+n//2][j+n//2] = -Hnby2[i][j]
        H[n] = Hn
        return Hn
    return H[n]

In [4]:
# Generating n boolean functions in k variables
def generate_data(k, n):
    two_pow_k = pow(2, k)
    # Input validation
    if(n<two_pow_k):
        print(f"Need 2^{k} inputs atleast for the model to converge. Give a bigger number for n")
        return []
    if(n>pow(2, two_pow_k)):
        print(f"n is greater than 2^2^{k}. Generating max possible functions - 2^2^{k}")
    if(two_pow_k<=16):
#         For generating data of sufficient LI inputs:
        while(True):
            perm = torch.randperm(pow(2, two_pow_k))[:n]
            data = [[0.0]*(two_pow_k-len(bin(num)[2:]))+[float(i) for i in bin(num)[2:]] for num in perm]
            data = [[1.0 if el==0.0 else -1.0 for el in bool_fun] for bool_fun in data]
            if(np.linalg.matrix_rank(data)<two_pow_k):
                print(f'Rank ({np.linalg.matrix_rank(data)}) of data is not large enough. Generating again.')
            else:
                print(f'Rank is {np.linalg.matrix_rank(data)}')
                break           
    else:
        # A maximum of 2^2^4 samples are generated (with samples) for any k>4
        data = [[1.0 if random.random()>0.5 else -1.0 for i in range(two_pow_k)] for i in range(min(n, pow(2, pow(2, 4))))]
    print("Data generated.")    
    return data            

In [5]:
# The network takes the processed k variable Boolean function (-1/1 for True/False) as the input and returns its Walsh Spectrum
# F -> WalshSpec
# WalshSpec = F * W_t

# forward(x) returns the Walsh spectrum of the Boolean function x, predict(x) return the non-linearity once the model is trained

class NetworkModel(nn.Module):

    def __init__(self, k):
        two_pow_k = pow(2, k)
        super(NetworkModel, self).__init__()
        self.lin1 = nn.Linear(two_pow_k, two_pow_k, bias=False)

    def forward(self, x):
        # Linear function without activation
        x = self.lin1(x)
        return x
    
    def predict(self, x):
        with torch.no_grad():
            self.eval()
            two_pow_k = len(x)
            nl = torch.round(0.5*(two_pow_k - max(self(x))))
        return nl
        

In [6]:
def train_network(model, data, numberOfInputs, learningRate, batchSize, epochs, device, optimizer):
    two_pow_k = len(data[0])
    n_ep, n_step = 0, 0
    for ep in range(0, epochs):
        for step in range(math.ceil(numberOfInputs/batchSize)):
            optimizer.zero_grad()
            input = torch.tensor(data[step:step+batchSize])
            output = model(input)
            loss = F.mse_loss(torch.matmul(input, Had(two_pow_k)), output)
            loss.backward()
            optimizer.step()
            weight_dict = OrderedDict(model.named_parameters())
            weightFunction = weight_dict['lin1.weight'].T
            H_error = F.mse_loss(Had(two_pow_k), weightFunction).item()
            if(H_error<0.00001):
                print(f"No. of epochs = {ep}, no. of steps = {step+ep*math.ceil(numberOfInputs/batchSize)}")
                n_ep, n_step = ep, step+ep*math.ceil(numberOfInputs/batchSize)
                return n_ep, n_step
    return -1, epochs*math.ceil(numberOfInputs/batchSize)

In [7]:
def test_network(model, data, n):
#     test_samples = torch.tensor([[float(el) for el in row] for row in torch.randint(0, 2, (n, self.inputDim)).to(self.device)])
    test_samples = torch.tensor(data[:-n])    
    preds = model(test_samples)
    true_spectra = torch.matmul(test_samples, Had(len(data[0])))
    print("Test:")
    for i in range(n):
        print(true_spectra[i], "\n", preds[i], "\n\n")

In [8]:
# Please change the number of variables (k) in the Boolean function here
k = 4
two_pow_k = pow(2, k)
n = pow(2, two_pow_k)

print(k, two_pow_k, n)

4 16 65536


In [9]:
##### Additional inputs may not be required to change.
learningRate = 4
batchSize = two_pow_k
numberOfInputs = n//4
max_epochs = 500

device = torch.device('cpu')
eps, steps = [], []
model = None
for i in range(1):
    data = generate_data(k, numberOfInputs)
    model =  NetworkModel(k)
    optimizer = optim.SGD(model.parameters(), lr=learningRate, momentum=0.9)
    n_ep, n_steps = train_network(model, data, numberOfInputs, learningRate, batchSize, max_epochs, device, optimizer)
    weightMatrix = OrderedDict(model.named_parameters())['lin1.weight'].T
    print("Final weight matrix: \n", weightMatrix)
    if(n_ep!=-1):
        eps.append(n_ep)
        steps.append(n_steps)
if(len(eps)!=0):
    print(f"Average number of epochs = {sum(eps)/len(eps)}, steps = {sum(steps)/len(steps)}.")
    if(len(eps)<10):
        print(f"Network converged well enough only {len(eps)} times out of 10")
else:
    print("Network did not converge well")
# test_network(model, data, 3)

Rank is 16
Data generated.
No. of epochs = 0, no. of steps = 330
Final weight matrix: 
 tensor([[ 1.0180,  1.0035,  0.9981,  0.9993,  1.0078,  1.0050,  1.0045,  0.9967,
          1.0112,  1.0077,  1.0043,  1.0070,  0.9925,  1.0076,  0.9989,  0.9987],
        [ 1.0007, -0.9995,  1.0001, -1.0004,  1.0002, -0.9996,  1.0003, -1.0000,
          0.9995, -0.9993,  1.0005, -0.9995,  1.0005, -0.9987,  1.0003, -1.0002],
        [ 1.0011,  1.0000, -1.0001, -0.9997,  1.0008,  1.0002, -0.9997, -0.9999,
          1.0009,  0.9999, -0.9998, -1.0000,  0.9985,  0.9999, -1.0004, -0.9999],
        [ 1.0063, -0.9987, -1.0005,  0.9997,  1.0029, -0.9981, -0.9984,  0.9991,
          1.0034, -0.9974, -0.9983,  1.0023,  0.9972, -0.9969, -1.0004,  0.9996],
        [ 1.0082,  1.0013,  0.9989,  1.0000, -0.9966, -0.9981, -0.9980, -1.0019,
          1.0064,  1.0033,  1.0017,  1.0034, -1.0037, -0.9978, -1.0009, -1.0006],
        [ 1.0028, -0.9996,  0.9997, -1.0000, -0.9982,  1.0012, -0.9994,  1.0002,
          1.0011

In [10]:
# Predict the non-linearity by passing the processed Boolean function of the form (-1)^f

model.predict(torch.tensor([1.0, -1.0, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1,-1]))

tensor(5.)